<a href="https://colab.research.google.com/github/ethngo7/poker-cv-project/blob/main/card_rank_suit_classifier_(poker_cv_project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ▸ setup
!pip install torch torchvision --quiet

from torchvision import datasets, transforms, models
import torch, torch.nn as nn, torch.optim as optim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00


In [1]:
!nvidia-smi

Mon Jul 14 23:47:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from google.colab import files
uploaded = files.upload() # choose kaggle zip


Saving kaggle - cards image dataset - kaggle.zip to kaggle - cards image dataset - kaggle.zip


In [4]:
import zipfile
import os

zip_path = "kaggle - cards image dataset - kaggle.zip"
extract_dir = "kaggle_individual_cards"   # this is where we'll extract it

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check the contents
print(os.listdir(extract_dir))


['train', 'cards.csv', 'valid', '53cards-53-(200 X 200)-100.00.h5', '14card types-14-(200 X 200)-94.61.h5', 'test']


In [5]:
# ▸ data
tfm = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder("kaggle_individual_cards/train", tfm)
val_ds   = datasets.ImageFolder("kaggle_individual_cards/valid",   tfm)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=64)

In [6]:
from tqdm.notebook import tqdm

# ▸ model
model = models.resnet18(weights="DEFAULT")
num_classes = len(train_ds.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# ▸ training loop (simple)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

NUM_EPOCHS = 5 #10

for epoch in range(5):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/10", leave=False)
    for x, y in pbar:
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({"loss": running_loss / (pbar.n + 1)})

torch.save(model.state_dict(), "card_classifier.pt")
print("✅ Training complete. Weights saved to 'card_classifier.pt'")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 67.9MB/s]


Epoch 1/10:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 4/10:   0%|          | 0/120 [00:00<?, ?it/s]

Epoch 5/10:   0%|          | 0/120 [00:00<?, ?it/s]

✅ Training complete. Weights saved to 'card_classifier.pt'


In [6]:
from google.colab import files
files.download("card_classifier.pt")